# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [40]:
import numpy as np
import pandas as pd
import requests as req
import math
from scipy import stats
import xlsxwriter

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [3]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [ ]:
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats/?token={IEX_CLOUD_API_TOKEN}'


## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [5]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [21]:
final_dataframe = pd.DataFrame(columns=my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote,stats&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = req.get(batch_api_call_url).json();
    for symbol in symbol_string.split(','):
        quote = data[symbol]['quote']
        stats = data[symbol]['stats']
        price = quote['latestPrice']
        oneYearReturn = stats['year1ChangePercent']
        series = pd.Series([symbol, price, oneYearReturn, 'N/A'],index = my_columns)
        final_dataframe = final_dataframe.append(series, ignore_index = True)

final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,144.29,0.625089,N/A
1,AAL,24.54,1.05451,N/A
2,AAP,190.79,0.345387,N/A
3,AAPL,126.14,0.594719,N/A
4,ABBV,117.50,0.316266,N/A
...,...,...,...,...
500,YUM,123.10,0.3726,N/A
501,ZBH,172.19,0.318054,N/A
502,ZBRA,505.33,0.970066,N/A
503,ZION,59.71,0.735482,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [22]:
final_dataframe.sort_values('One-Year Price Return', ascending = False, inplace = True)
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace = True)
final_dataframe

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,179,FCX,43.45,3.70244,N/A
1,275,LB,71.55,3.03827,N/A
2,253,IVZ,29.39,2.59412,N/A
3,208,GPS,34.12,2.40711,N/A
4,441,TPR,46.33,1.93734,N/A
5,314,MOS,36.48,1.89858,N/A
6,410,SIVB,610.17,1.66918,N/A
7,385,PWR,99.95,1.61847,N/A
8,233,HWM,36.39,1.58244,N/A
9,272,KSS,56.94,1.56719,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [31]:
portfolio_size = 10000/len(final_dataframe.index)
for i in range(0, len(final_dataframe.index)):
    final_dataframe.loc[i, 'ToBuy'] = math.floor(portfolio_size/final_dataframe.loc[i, 'Price'])


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [34]:
hqm_columns = [
    'ticker',
    'price',
    '1yr',
    '6mo',
    '3mo',
    '1mo',
    '1yr_percent',
    '6mo_percent',
    '3mo_percent',
    '1mo_percent',
     'to_buy',
    
]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote,stats&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = req.get(batch_api_call_url).json();
    for symbol in symbol_string.split(','):
        quote = data[symbol]['quote']
        stats = data[symbol]['stats']
        price = quote['latestPrice']
        one_yr = stats['year1ChangePercent']
        six_mo = stats['month6ChangePercent']
        three_mo = stats['month3ChangePercent']
        one_mo = stats['month1ChangePercent']
        series = pd.Series([symbol, price, one_yr, six_mo, three_mo, one_mo, 0,0,0,0,0],index = hqm_columns)
        hqm_dataframe = hqm_dataframe.append(series, ignore_index = True)

hqm_dataframe

,ticker,price,1yr,6mo,3mo,1mo,1yr_percent,6mo_percent,3mo_percent,1mo_percent,to_buy
0,A,143.43,0.622918,0.212292,0.129939,0.0080925,0,0,0,0,0
1,AAL,24.36,1.05359,0.632515,0.170321,0.127466,0,0,0,0,0
2,AAP,192.89,0.341347,0.273441,0.178088,-0.0449793,0,0,0,0,0
3,AAPL,130.07,0.596476,0.0783566,0.0356174,-0.0676684,0,0,0,0,0
4,ABBV,113.30,0.318197,0.0981329,0.0575687,0.00794692,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,121.60,0.378286,0.13088,0.170723,0.0374746,0,0,0,0,0
501,ZBH,175.80,0.32381,0.123676,0.0304719,-0.0508507,0,0,0,0,0
502,ZBRA,497.66,0.963798,0.330327,-0.00268137,-0.00732282,0,0,0,0,0
503,ZION,58.56,0.71487,0.493547,0.0981193,0.058056,0,0,0,0,0


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [51]:
time_periods = ['1yr', '6mo', '3mo', '1mo']
for time_period in time_periods:
    time_period_percent = time_period + '_percent'
    alldata = hqm_dataframe[time_period]
    for row in hqm_dataframe.index:
        value = hqm_dataframe.loc[row, time_period]
        if value == None:
            hqm_dataframe.loc[row, time_period] = 0
        
for time_period in time_periods:
    time_period_percent = time_period + '_percent'
    alldata = hqm_dataframe[time_period]
    for row in hqm_dataframe.index:
        value = float(hqm_dataframe.loc[row, time_period])
        percent = stats.percentileofscore(alldata, value)
        hqm_dataframe.loc[row, time_period_percent] = percent


hqm_dataframe        

,ticker,price,1yr,6mo,3mo,1mo,1yr_percent,6mo_percent,3mo_percent,1mo_percent,to_buy
0,A,143.43,0.622918,0.212292,0.129939,0.0080925,66.5347,50.495,45.3465,44.7525,0
1,AAL,24.36,1.05359,0.632515,0.170321,0.127466,90.6931,95.6436,62.1782,95.6436,0
2,AAP,192.89,0.341347,0.273441,0.178088,-0.0449793,38.6139,63.3663,65.9406,14.0594,0
3,AAPL,130.07,0.596476,0.0783566,0.0356174,-0.0676684,65.1485,23.5644,18.6139,7.92079,0
4,ABBV,113.30,0.318197,0.0981329,0.0575687,0.00794692,34.8515,27.7228,24.9505,44.3564,0
...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,121.60,0.378286,0.13088,0.170723,0.0374746,43.1683,35.6436,62.5743,63.1683,0
501,ZBH,175.80,0.32381,0.123676,0.0304719,-0.0508507,35.6436,33.6634,17.6238,12.2772,0
502,ZBRA,497.66,0.963798,0.330327,-0.00268137,-0.00732282,88.1188,73.8614,9.50495,31.8812,0
503,ZION,58.56,0.71487,0.493547,0.0981193,0.058056,73.6634,90.099,35.8416,74.6535,0


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [55]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentimes = []
    for time_period in time_periods:
        time_period_percent = time_period + '_percent'
        momentum_percentimes.append(hqm_dataframe.loc[row, time_period_percent])
    hqm_dataframe.loc[row, 'hqm_score'] = mean(momentum_percentimes)


hqm_dataframe.sort_values('hqm_score', ascending = False, inplace=True)
hqm_dataframe = hqm_dataframe[:50]
hqm_dataframe.reset_index(inplace = True)

hqm_dataframe

,index,ticker,price,1yr,6mo,3mo,1mo,1yr_percent,6mo_percent,3mo_percent,1mo_percent,to_buy,hqm_score
0,344,NUE,106.36,1.43503,0.86014,0.707547,0.25358,96.6337,99.2079,100,100,0,98.960396
1,149,DXC,38.87,1.20177,0.677319,0.547006,0.213363,94.2574,96.8317,99.802,99.2079,0,97.524752
2,106,COF,165.18,1.33437,0.865857,0.353259,0.168933,95.4455,99.604,97.2277,97.2277,0,97.376238
3,174,F,15.09,1.53576,0.65572,0.279526,0.200744,98.0198,96.2376,92.2772,98.4158,0,96.237624
4,179,FCX,43.81,3.68549,0.854141,0.265121,0.104824,100,99.0099,89.703,92.4752,0,95.297030
5,148,DVN,27.01,1.29656,0.864841,0.253506,0.206441,94.8515,99.4059,87.3267,98.8119,0,95.099010
6,450,TXT,69.06,1.20548,0.472669,0.372632,0.123098,94.4554,88.7129,98.0198,94.8515,0,94.009901
7,253,IVZ,28.77,2.6139,0.700125,0.277054,0.0837663,99.604,97.6238,91.4851,84.7525,0,93.366337
8,218,HES,87.89,0.79324,0.720055,0.308795,0.21705,79.604,98.2178,96.0396,99.4059,0,93.316832
9,60,BEN,35.38,0.882999,0.584915,0.329718,0.152987,84.9505,94.6535,96.6337,96.4356,0,93.168317


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [ ]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

## Saving Our Excel Output

As before, saving our Excel output is very easy: